# SQL Portfolio Project – Retail Store Analysis

This notebook is a SQL portfolio project created to demonstrate my ability to write clear and correct SQL queries for data exploration and analysis.

The analysis is performed on a retail store dataset and focuses on:

- Selecting and filtering data

- Using aggregate functions to summarize information

- Grouping data to answer business-related questions

- Applying sorting and conditions to extract meaningful results

All queries are written with readability and correctness in mind, reflecting how SQL is commonly used for basic to intermediate analysis tasks in real-world environments.

This notebook was created as part of my SQL portfolio for job applications and is intended to showcase foundational SQL skills applied to a practical dataset.

## Creating a database from the CSV file using sqlite

In [1]:
import pandas as pd
import sqlite3
from pathlib import Path
csv_path = Path("retail_sales_dataset.csv")       
db_path = Path("retail_sales.db") 

In [2]:
df = pd.read_csv(csv_path)


In [3]:
print('original column:', df.columns.tolist())

original column: ['Transaction ID', 'Date', 'Customer ID', 'Gender', 'Age', 'Product Category', 'Quantity', 'Price per Unit', 'Total Amount']


In [4]:
df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

In [5]:
print(f"cleaned columns: {df.columns.tolist()}")

cleaned columns: ['transaction_id', 'date', 'customer_id', 'gender', 'age', 'product_category', 'quantity', 'price_per_unit', 'total_amount']


In [6]:
#connect or create an sqlite database
conn = sqlite3.connect(db_path)

In [7]:
#import dataframe to sql
df.to_sql(name= "retail_sales", con=conn, if_exists = 'replace', index = False)

1000

In [8]:
#check the number of rows and columns imported
row_count = pd.read_sql("SELECT COUNT(*) As cnt FROM retail_sales;", conn)
row_count


,cnt
0,1000


In [9]:
print(f"Rows imported: {int(row_count['cnt'][0])}")

Rows imported: 1000


In [10]:
col_count = pd.read_sql("PRAGMA table_info(retail_sales);", conn)
len(col_count)

9

In [11]:
 pd.read_sql("PRAGMA table_info(retail_sales);", conn)['name']

0      transaction_id
1                date
2         customer_id
3              gender
4                 age
5    product_category
6            quantity
7      price_per_unit
8        total_amount
Name: name, dtype: object

In [12]:
conn.close()

In [13]:
print(f"\nDone! SQLite database created at: {db_path.resolve()}")


Done! SQLite database created at: C:\Users\chinw\Documents\SQL_projects\retail_sales.db


In [14]:
#load extension and connect to sqlite
%load_ext sql

In [15]:
%sql sqlite:///retail_sales.db

'Connected: @retail_sales.db'

In [16]:
%sql SELECT * FROM retail_sales LIMIT 5;


 * sqlite:///retail_sales.db
Done.


transaction_id,date,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount
1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [17]:
#Total revenue

In [18]:
%%sql

SELECT
    SUM(total_amount) AS total_revenue
FROM retail_sales;

 * sqlite:///retail_sales.db
Done.


total_revenue
456000


In [19]:
#total number of transactions

In [20]:
%%sql

SELECT
     COUNT(*) AS total_transactions
FROM retail_sales;

 * sqlite:///retail_sales.db
Done.


total_transactions
1000


In [21]:
#total quantity sold

In [22]:
%%sql

SELECT
    SUM(quantity) AS total_quantity_sold
FROM retail_sales;

 * sqlite:///retail_sales.db
Done.


total_quantity_sold
2514


In [23]:
#Average order value

In [24]:
%%sql

SELECT 
    SUM(total_amount) / COUNT(*) * 1.0 AS average_order_value
FROM retail_sales;

 * sqlite:///retail_sales.db
Done.


average_order_value
456.0


In [25]:
#Revenue by product category

In [26]:
%%sql

SELECT product_category, SUM(total_amount) AS revenue
FROM retail_sales
GROUP BY product_category
ORDER BY revenue DESC;

 * sqlite:///retail_sales.db
Done.


product_category,revenue
Electronics,156905
Clothing,155580
Beauty,143515


In [27]:
#Time-based Analysis(Monthly trends)

In [28]:
#monthly revenue

In [29]:
%%sql

SELECT 
    strftime('%Y-%m',date) AS month,
    SUM(total_amount) AS monthly_revenue
FROM retail_sales
GROUP BY month
ORDER BY monthly_revenue;

 * sqlite:///retail_sales.db
Done.


month,monthly_revenue
2024-01,1530
2023-09,23620
2023-03,28990
2023-04,33870
2023-11,34920
2023-01,35450
2023-07,35465
2023-06,36715
2023-08,36960
2023-02,44060


#transactions_per_month

In [30]:
%%sql

SELECT 
   strftime('%Y-%m', date) as month,
    COUNT(*) as transaction_count
FROM retail_sales
GROUP BY month
ORDER BY month;

 * sqlite:///retail_sales.db
Done.


month,transaction_count
2023-01,76
2023-02,85
2023-03,73
2023-04,86
2023-05,105
2023-06,77
2023-07,72
2023-08,94
2023-09,65
2023-10,96


In [31]:
#Month-over-Month revenue growth(%)

In [32]:
%%sql
WITH monthly_revenue AS (
    SELECT
        strftime('%Y-%m', date) AS month,
        SUM(total_amount) AS revenue
    FROM retail_sales
    GROUP BY month
),
revenue_with_lag AS (
    SELECT
        month,
        revenue,
        LAG(revenue) OVER (ORDER BY month) AS prev_revenue
    FROM monthly_revenue
)
SELECT
    month,
    revenue,
    prev_revenue,
    CASE 
        WHEN prev_revenue IS NULL THEN NULL
        ELSE ROUND( (revenue - prev_revenue) * 100.0 / prev_revenue, 2)
    END AS month_growth_percent
FROM revenue_with_lag
ORDER BY month;




 * sqlite:///retail_sales.db
Done.


month,revenue,prev_revenue,month_growth_percent
2023-01,35450,None,None
2023-02,44060,35450,24.29
2023-03,28990,44060,-34.2
2023-04,33870,28990,16.83
2023-05,53150,33870,56.92
2023-06,36715,53150,-30.92
2023-07,35465,36715,-3.4
2023-08,36960,35465,4.22
2023-09,23620,36960,-36.09
2023-10,46580,23620,97.21


In [33]:
#customer Analysis

In [34]:
#3.1 Customer lifetime value (total spend per customer)

In [35]:
%%sql
SELECT
    customer_id,
    SUM(total_amount) AS total_spent,
    COUNT(*) AS total_orders
FROM retail_sales
GROUP BY customer_id
ORDER BY total_spent DESC
LIMIT 5;


 * sqlite:///retail_sales.db
Done.


customer_id,total_spent,total_orders
CUST970,2000,1
CUST946,2000,1
CUST927,2000,1
CUST875,2000,1
CUST832,2000,1


In [36]:
#3.2 Top 10 customers by spend


In [37]:
%%sql
SELECT
    customer_id,
    SUM(total_amount) AS total_spent
FROM retail_sales
GROUP BY customer_id
ORDER BY total_spent DESC
LIMIT 5;


 * sqlite:///retail_sales.db
Done.


customer_id,total_spent
CUST970,2000
CUST946,2000
CUST927,2000
CUST875,2000
CUST832,2000


In [38]:
#Revenue by gender

In [39]:
%%sql
SELECT
    gender,
    SUM(total_amount) AS total_revenue,
    COUNT(*) AS total_transactions
FROM retail_sales
GROUP BY gender;

 * sqlite:///retail_sales.db
Done.


gender,total_revenue,total_transactions
Female,232840,510
Male,223160,490


In [40]:
#3.4 Revenue by age group (segmentation with CASE)

In [41]:
%%sql
SELECT
    CASE
        WHEN age BETWEEN 18 AND 25 THEN '18-25'
        WHEN age BETWEEN 26 AND 35 THEN '26-35'
        WHEN age BETWEEN 36 AND 50 THEN '36-50'
        ELSE '51+'
    END AS age_group,
    SUM(total_amount) AS total_revenue,
    COUNT(*) AS total_transactions
FROM retail_sales
GROUP BY age_group
ORDER BY total_revenue DESC;


 * sqlite:///retail_sales.db
Done.


age_group,total_revenue,total_transactions
36-50,139660,313
51+,133310,313
26-35,98480,205
18-25,84550,169


In [42]:
#Product Category Analysis

In [43]:
#4.1 Quantity sold per product category

In [44]:
%%sql
SELECT
    product_category,
    SUM(quantity) AS total_quantity_sold
FROM retail_sales
GROUP BY product_category
ORDER BY total_quantity_sold DESC;


 * sqlite:///retail_sales.db
Done.


product_category,total_quantity_sold
Clothing,894
Electronics,849
Beauty,771


In [45]:
#4.2 Average order value by product category

In [46]:
%%sql
SELECT
    product_category,
    SUM(total_amount) * 1.0 / COUNT(*) AS avg_order_value
FROM retail_sales
GROUP BY product_category
ORDER BY avg_order_value DESC;


 * sqlite:///retail_sales.db
Done.


product_category,avg_order_value
Beauty,467.4755700325733
Electronics,458.7865497076023
Clothing,443.2478632478632


In [47]:
#4.3 Rank categories by revenue (window function)

In [48]:
%%sql
SELECT
    product_category,
    SUM(total_amount) AS total_revenue,
    RANK() OVER (ORDER BY SUM(total_amount) DESC) AS revenue_rank
FROM retail_sales
GROUP BY product_category
ORDER BY revenue_rank;


 * sqlite:///retail_sales.db
Done.


product_category,total_revenue,revenue_rank
Electronics,156905,1
Clothing,155580,2
Beauty,143515,3


In [49]:
#5️⃣ Advanced SQL (CTEs, window functions, segmentation)

In [50]:
#5.1 Rank customers by spend within each product category

In [51]:
%%sql
SELECT
    product_category,
    customer_id,
    SUM(total_amount) AS customer_spend,
    RANK() OVER (
        PARTITION BY product_category 
        ORDER BY SUM(total_amount) DESC
    ) AS spend_rank_in_category
FROM retail_sales
GROUP BY product_category, customer_id
ORDER BY product_category, spend_rank_in_category
LIMIT 10;


 * sqlite:///retail_sales.db
Done.


product_category,customer_id,customer_spend,spend_rank_in_category
Beauty,CUST832,2000,1
Beauty,CUST808,2000,1
Beauty,CUST743,2000,1
Beauty,CUST592,2000,1
Beauty,CUST577,2000,1
Beauty,CUST503,2000,1
Beauty,CUST480,2000,1
Beauty,CUST447,2000,1
Beauty,CUST281,2000,1
Beauty,CUST257,2000,1


In [52]:
#5.2 Top 5 customers per month

In [53]:
%%sql
WITH customer_monthly_spend AS (
    SELECT
        strftime('%Y-%m', date) AS month,
        customer_id,
        SUM(total_amount) AS total_spent
    FROM retail_sales
    GROUP BY month, customer_id
),
ranked_customers AS (
    SELECT
        month,
        customer_id,
        total_spent,
        RANK() OVER (
            PARTITION BY month
            ORDER BY total_spent DESC
        ) AS spend_rank
    FROM customer_monthly_spend
)
SELECT
    month,
    customer_id,
    total_spent
FROM ranked_customers
WHERE spend_rank <= 5
ORDER BY month, total_spent DESC
LIMIT 10;


 * sqlite:///retail_sales.db
Done.


month,customer_id,total_spent
2023-01,CUST015,2000
2023-01,CUST420,2000
2023-01,CUST592,2000
2023-01,CUST742,2000
2023-01,CUST743,2000
2023-02,CUST152,2000
2023-02,CUST257,2000
2023-02,CUST269,2000
2023-02,CUST416,2000
2023-02,CUST577,2000


In [54]:
# 5.3 Segment customers into High / Medium / Low value

In [55]:
%%sql
WITH customer_spend AS (
    SELECT
        customer_id,
        SUM(total_amount) AS total_spent
    FROM retail_sales
    GROUP BY customer_id
)
SELECT
    customer_id,
    total_spent,
    CASE
        WHEN total_spent >= 2000 THEN 'High value'
        WHEN total_spent BETWEEN 1000 AND 1999 THEN 'Medium value'
        ELSE 'Low value'
    END AS customer_segment
FROM customer_spend
ORDER BY total_spent ASC
LIMIT 10;


 * sqlite:///retail_sales.db
Done.


customer_id,total_spent,customer_segment
CUST044,25,Low value
CUST103,25,Low value
CUST185,25,Low value
CUST191,25,Low value
CUST201,25,Low value
CUST204,25,Low value
CUST205,25,Low value
CUST206,25,Low value
CUST223,25,Low value
CUST230,25,Low value


In [56]:
#5.4 7-day rolling revenue

In [57]:
%%sql
WITH daily_revenue AS (
    SELECT
        date,
        SUM(total_amount) AS daily_revenue
    FROM retail_sales
    GROUP BY date
)
SELECT
    date,
    daily_revenue,
    SUM(daily_revenue) OVER (
        ORDER BY date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS rolling_7_day_revenue
FROM daily_revenue
ORDER BY date;


 * sqlite:///retail_sales.db
Done.


date,daily_revenue,rolling_7_day_revenue
2023-01-01,3600,3600
2023-01-02,1765,5365
2023-01-03,600,5965
2023-01-04,1240,7205
2023-01-05,1100,8305
2023-01-06,620,8925
2023-01-07,150,9075
2023-01-08,625,6100
2023-01-09,200,4535
2023-01-10,230,4165


In [58]:
#6.1 Show table schema

In [59]:
%%sql
PRAGMA table_info(retail_sales);


 * sqlite:///retail_sales.db
Done.


cid,name,type,notnull,dflt_value,pk
0,transaction_id,INTEGER,0,None,0
1,date,TEXT,0,None,0
2,customer_id,TEXT,0,None,0
3,gender,TEXT,0,None,0
4,age,INTEGER,0,None,0
5,product_category,TEXT,0,None,0
6,quantity,INTEGER,0,None,0
7,price_per_unit,INTEGER,0,None,0
8,total_amount,INTEGER,0,None,0


In [60]:
#count columns

In [61]:
%%sql
SELECT COUNT(*) AS num_columns
FROM pragma_table_info('retail_sales');


 * sqlite:///retail_sales.db
Done.


num_columns
9
